## MVPC

This notebook was created by 
* Heejung Jung
* Xiaochun Han
* Deepanshi 

In [163]:
# load libraries
import os
import numpy as np
import mvpa2
import pandas as pd
from mvpa2.tutorial_suite import *
import nilearn
import nipy
import mvpa2.suite as mv
import nibabel

# 1 out of 20

## load behavioral data

In [115]:
# create two column text file
main_dir = '/Users/h/Documents/projects_local/haxby_mvpc'
# data_path = os.path.join(tutorial_data_path, 'haxby2001')
attr_tsv = os.path.join(main_dir, 'data','attention','sub-rid000001_task-beh_run-1_events.tsv')
attr = pd.read_csv(attr_tsv, sep='\t')
attr['chunks'] = np.repeat([0,1], 44)

In [116]:
attr.head()

,onset,duration,trial_type,taxonomy,behavior,task,repetition,response_time,mirror,stim_file,chunks
0,12,2,ungulate_running,ungulate,running,behavior,none,none,no,ungulate_running_2.mp4,0
1,16,2,bird_fighting,bird,fighting,behavior,none,none,yes,bird_fighting_1.mp4,0
2,20,2,insect_swimming,insect,swimming,behavior,none,none,yes,insect_swimming_2.mp4,0
3,28,2,bird_eating,bird,eating,behavior,none,none,yes,bird_eating_1.mp4,0
4,32,2,ungulate_eating,ungulate,eating,behavior,behavior,0.248026133,no,ungulate_eating_1.mp4,0


In [117]:
attr_subset = attr[['trial_type', 'chunks']]

In [118]:
attr_subset.rename(columns={'trial_type':'targets'}, 
                 inplace=True)
# pd.concat([new_attr]*2, ignore_index=True)

## running glm

In [188]:
gifti_fname = os.path.join(main_dir, 'attention','derivatives','fmriprep','sub-rid000001','func','sub-rid000001_task-beh_run-1_bold_space-fsaverage6.L.func.gii')
nifti_fname= '/Users/h/Documents/projects_local/haxby_mvpc/attention/derivatives/sub-rid000001/sub-rid000001_task-beh_run-1_bold.nii.gz'
#surf = mv.surf.read(gifti_fname)
ds = fmri_dataset(nifti_fname)
#qe = mv.SurfaceQueryEngine(surf, 10.0, distance_metric='dijkstra')

# model_gen = lambda y, x: sm.OLS(y, x)

In [120]:
events = find_events(targets=attr.trial_type,
                     chunks = attr.chunks)

In [121]:
TR = 2
for ind, ev in enumerate(events):
    ev['onset'] = attr['onset'][ind]
    ev['duration'] = 2

In [150]:
# glm
evds = fit_event_hrf_model(ds, events,
                           time_attr = 'time_coords',
                           condition_attr=('targets','chunks'))

In [166]:
# Z-score features across samples
evds.samples = ((evds.samples - np.mean(evds.samples, axis=1)[:, None])
              / np.std(evds.samples, axis=1)[:, None])

## classification

In [179]:
# classifier
clf = LinearCSVMC()
cv = CrossValidation(clf, NFoldPartitioner())


In [180]:
cv_glm = cv(evds)

In [182]:
np.mean(cv_glm)

0.9473684210526316

### searchlight doesn't work - work on it

In [ ]:
# search light
# sl = Searchlight(cv,
#  IndexQueryEngine(voxel_indices=Sphere(1),
#  event_offsetidx=Sphere(2)),
#  postproc=mean_sample())
qe = mv.SurfaceQueryEngine(surf, 10.0, distance_metric='dijkstra')

sl = Searchlight(cv, 
                 queryengine=qe, 
                 enable_ca=['roi_sizes'],
                    nproc=4, 
                 roi_ids=cortical_vertices)
res = sl(evds)

## kNN

In [154]:
clf = kNN(k=1, dfx=one_minus_correlation, voting='majority')
cv = CrossValidation(clf, NFoldPartitioner(attr='chunks'))
cv_glm = cv(evds)
print '%.2f' % np.mean(cv_glm)

0.95


In [29]:
ds.sa.time_indices[:5] # array([0, 1, 2, 3, 4])

ds.sa.time_coords[:5] # array([0., 2., 4., 6., 8.])

ds.fa.voxel_indices[:5]
# array([[0, 0, 0],
#        [0, 0, 1],
#        [0, 0, 2],
#        [0, 0, 3],
#        [0, 0, 4]])

ds.a.voxel_eldim # (3.0, 3.0, 3.0)

ds.a.voxel_dim # (80, 80, 42)

'imghdr' in ds.a # True

True

In [30]:
print ds.a.mapper

<FlattenMapper>


# classify taxa (1 out of 5), training on videos with 3 behaviors and testing on videos with the left-out behavior.

In [ ]:
# split data training on "non-eating" testing on "eating"
ds_split1 = ds[ds.sa.taxa != 'eating']
len(ds_split1)
ds_split2 = ds[ds.sa.runtype == 'eating']
len(ds_split2)


# Classify behaviors (1 out of 4), training on videos with 4 taxa and testing on videos with left-out taxonomic category